In [2]:
import pandas as pd
import requests
import sqlite3
from config import settings

In [3]:
connection = sqlite3.connect(settings.db_name, check_same_thread = False)

In [58]:
def get_historical_data(ticker, output_size="full"):
    """
    A function that downloads the historical data from Alpha Vantage website using free API key.

    Parameters required:
    -------------------
    Ticker - The stock symbol
    Output_size  - The two options available are full or compact. The function is set to full by default.
    """

    url = ("https://www.alphavantage.co/query?"
            "function=TIME_SERIES_DAILY_ADJUSTED&"
            f"symbol={ticker}&"
            f"outputsize={output_size}&"
            f"apikey={settings.alpha_api_key}"
         )
    response = requests.get(url=url) 
    #Convert the response to json
    response_json = response.json()
    #Extract the Daily Time series
    data_dict = response_json["Time Series (Daily)"]
    #Convert the data dictionary into DataFrame
    df = pd.DataFrame.from_dict(data_dict, orient = "index", dtype = float)
    #Convert the index to datetime and set the name to date
    df.index = pd.to_datetime(df.index)
    df.index.name = "date"
    #Clean the column names
    df.columns = [c.split(". ")[1] for c in df.columns] 
    #Drop the unwanted columns
    df = df.drop(["adjusted close", "volume", "dividend amount", "split coefficient"], axis = 1)
    return df

In [6]:
from data import AlphaVantageApi

ticker = "XLK"
api = AlphaVantageApi()
xlk_data = api.get_historical_data(ticker=ticker, output_size="compact")
print(xlk_data.info())
xlk_data.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 100 entries, 2023-01-06 to 2022-08-16
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    100 non-null    float64
 1   high    100 non-null    float64
 2   low     100 non-null    float64
 3   close   100 non-null    float64
dtypes: float64(4)
memory usage: 3.9 KB
None


,open,high,low,close
date,,,,
2023-01-06,122.25,125.29,120.81,124.73
2023-01-05,122.96,123.28,120.97,121.18
2023-01-04,123.90,124.37,122.22,123.58
2023-01-03,125.77,126.60,122.38,123.26
2022-12-30,123.32,124.51,122.55,124.44


In [7]:
from data import SQLRepository

Repo = SQLRepository(connection=connection)

In [8]:
Repo.insert_table(table_name=ticker, records=xlk_data, if_exists = "replace")

{'Transaction successful': True, 'Number of records inserted': 100}

In [9]:
df = Repo.read_table(table_name=ticker, limit = 50)
print(df.head())
len(df)

              open    high     low   close
date                                      
2023-01-06  122.25  125.29  120.81  124.73
2023-01-05  122.96  123.28  120.97  121.18
2023-01-04  123.90  124.37  122.22  123.58
2023-01-03  125.77  126.60  122.38  123.26
2022-12-30  123.32  124.51  122.55  124.44


50

In [3]:
import arch

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
import warnings

warnings.filterwarnings("ignore")